# **le corpus de référence en chinois**

1)现代汉语常用词表 2014.txt

In [1]:
# 文件路径
input_file = "/content/现代汉语常用词表 2014.txt"
output_file = "/content/现代汉语常用词表 2014_new.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        # 按制表符分隔，只取第一个元素
        parts = line.strip().split("\t")
        if parts:  # 防止空行
            fout.write(parts[0] + "\n")

print("提取完成！输出文件为：", output_file)


FileNotFoundError: [Errno 2] No such file or directory: '/content/现代汉语常用词表 2014.txt'

2) 2016,现代汉语第七版.txt

In [2]:
import re

# 输入输出路径
input_file = "/content/2016,现代汉语第七版.txt"
output_file = "/content/2016,现代汉语第七版_new.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        # 用正则提取【】中的内容
        match = re.search(r"【([^】]+)】", line)
        if match:
            word = match.group(1)
            fout.write(word + "\n")

print("提取完成！输出文件为：", output_file)


FileNotFoundError: [Errno 2] No such file or directory: '/content/2016,现代汉语第七版.txt'

3) 2009现代汉语词典

In [ ]:
import re

# 文件路径
input_file = "/content/2009上册.txt"
output_file = "/content/2009上册_new.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        parts = line.strip().split("\t")
        if len(parts) >= 3:
            # 第三列是汉字词条，去掉非汉字字符（保留中文）
            hanzi = re.sub(r"[^\u4e00-\u9fff]", "", parts[2])
            if hanzi:
                fout.write(hanzi + "\n")

print("提取完成！输出文件为：", output_file)


提取完成！输出文件为： /content/2009上册_new.txt


In [ ]:
import re

# 文件路径
input_file = "/content/2009下册.txt"
output_file = "/content/2009下册_new.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        parts = line.strip().split("\t")
        if len(parts) >= 3:
            # 第三列是汉字词条，去掉非汉字字符（保留中文）
            hanzi = re.sub(r"[^\u4e00-\u9fff]", "", parts[2])
            if hanzi:
                fout.write(hanzi + "\n")

print("提取完成！输出文件为：", output_file)


提取完成！输出文件为： /content/2009下册_new.txt


维基百科词典


In [3]:
import requests
import json
from datetime import datetime

API_URL = "https://zh.wiktionary.org/w/api.php"
CUTOFF_DATE = datetime(2015, 1, 1)
OUTPUT_FILE = "wiktionary_zh_pre2015.jsonl"

titles = []
apcontinue = None

print("开始抓取中文维基词典标题...")

# 1. 获取所有主命名空间的标题
while True:
    params = {
        "action": "query",
        "list": "allpages",
        "apnamespace": 0,   # 主命名空间
        "aplimit": "500",
        "format": "json"
    }
    if apcontinue:
        params["apcontinue"] = apcontinue

    r = requests.get(API_URL, params=params)
    data = r.json()

    for page in data["query"]["allpages"]:
        titles.append(page["title"])

    if "continue" in data:
        apcontinue = data["continue"]["apcontinue"]
    else:
        break

print(f"总共抓到 {len(titles)} 个词条标题，开始过滤 2015 年以前的...")

# 2. 过滤出 2015 年以前创建的词条
pre2015_titles = []
for i, title in enumerate(titles, 1):
    params = {
        "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvlimit": 1,
        "rvdir": "newer",  # 最早修订
        "rvprop": "timestamp",
        "format": "json"
    }
    r = requests.get(API_URL, params=params)
    pages = r.json()["query"]["pages"]

    for _, page_data in pages.items():
        if "revisions" in page_data:
            ts = page_data["revisions"][0]["timestamp"]
            created_time = datetime.fromisoformat(ts.replace("Z", "+00:00"))
            if created_time < CUTOFF_DATE:
                pre2015_titles.append(title)

    if i % 500 == 0:
        print(f"已检查 {i}/{len(titles)}...")

# 3. 写入 JSONL 文件
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for t in sorted(pre2015_titles):
        f.write(json.dumps({"text": t, "source": "wiktionary"}, ensure_ascii=False) + "\n")

print(f"完成！共保存 {len(pre2015_titles)} 个 2015 年以前的中文 Wiktionary 词条到 {OUTPUT_FILE}")


开始抓取中文维基词典标题...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#Fusionner le corpus

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#步骤 1：将所有 CSV 文件合并成一个文件

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# 递归合并（带分隔符/编码自适应）+ 详尽报告
import pandas as pd
import os, time, re
from tqdm import tqdm

start = time.time()

FOLDER = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/ch_corpus'
OUT    = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_all_data_2015_2025.csv'
REPORT = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report.csv'

REQUIRED_COLS = ['author','title','date','content','url']  # 你的目标列

def read_csv_robust(path):
    """返回 (df, used_sep, used_enc, err)"""
    seps_try = [None, ',', ';', '\t', '|']   # None=自动嗅探（engine='python'）
    encs_try = ['utf-8', 'utf-8-sig', 'latin1']
    last_err = None
    for enc in encs_try:
        for sep in seps_try:
            try:
                df = pd.read_csv(
                    path,
                    sep=sep,
                    engine='python',
                    encoding=enc,
                    on_bad_lines='skip'
                )
                return df, (sep if sep is not None else 'auto'), enc, None
            except Exception as e:
                last_err = str(e)
                continue
    return pd.DataFrame(), 'fail', 'fail', last_err

if not os.path.exists(FOLDER):
    raise FileNotFoundError(f"路径不存在: {FOLDER}")

# 收集 CSV
csv_files = []
for root, _, files in os.walk(FOLDER):
    for f in files:
        if f.lower().endswith('.csv'):
            csv_files.append(os.path.join(root, f))
print(f"共找到 {len(csv_files)} 个 CSV 文件\n")

# 读取 + 报告
records = []
dfs = []

for fp in tqdm(csv_files, desc="读取 CSV", unit="文件"):
    df, used_sep, used_enc, err = read_csv_robust(fp)

    # 标注来源、路径年份提示
    source = 'unknown'
    for s in ['Kenh','Genk','Cafebize']:
        if re.search(rf'/{s}/', fp, flags=re.I):
            source = s.lower()
            break
    y_hint = None
    y = re.search(r'(20[1-2]\d)', fp)
    if y: y_hint = y.group(1)

    # 统一列名（去空格小写），避免“Date”“URL”之类大小写不一致
    old_cols = df.columns.tolist()
    df.columns = [c.strip() for c in df.columns]
    lower_map = {c: c.lower() for c in df.columns}
    df.rename(columns=lower_map, inplace=True)

    row_count = len(df)
    has_cols = {c: (c in df.columns) for c in REQUIRED_COLS}

    records.append({
        'file': fp,
        'rows': row_count,
        'used_sep': used_sep,
        'used_encoding': used_enc,
        'has_author': has_cols['author'],
        'has_title': has_cols['title'],
        'has_date': has_cols['date'],
        'has_content': has_cols['content'],
        'has_url': has_cols['url'],
        'source': source,
        'year_hint': y_hint,
        'error': '' if err is None else err[:120]
    })

    # 只要不是“完全空表”就先收进来；后续再统一处理日期
    if row_count > 0:
        # 缺的列补空，确保结构统一
        for c in REQUIRED_COLS:
            if c not in df.columns:
                df[c] = None
        df['source'] = source
        df['year_hint'] = y_hint
        dfs.append(df[REQUIRED_COLS + ['source','year_hint']])

# 输出报告
report_df = pd.DataFrame.from_records(records)
report_df.to_csv(REPORT, index=False)
print(f" 读取报告已保存：{REPORT}")
print("—— Top 5 读取最少行的文件 ——")
print(report_df.sort_values('rows').head(5)[['file','rows','used_sep','used_encoding','error']])

if not dfs:
    raise ValueError("所有文件都读空了，请检查报告 merge_report.csv")

# 合并
all_data = pd.concat(dfs, ignore_index=True)
print(f"合并后总行数：{len(all_data):,}")

# 日期解析（尽量保留；只对成功解析的行做年份过滤，失败的行也保留）
def try_parse_date(s):
    if pd.isna(s): return pd.NaT
    s = str(s).strip()
    for fmt in ("%d/%m/%Y %I:%M %p","%d/%m/%Y %H:%M","%d/%m/%Y",
                "%Y-%m-%d %H:%M:%S","%Y-%m-%d %H:%M","%Y-%m-%d",
                "%Y/%m/%d %H:%M:%S","%Y/%m/%d"):
        try: return pd.to_datetime(s, format=fmt)
        except: pass
    return pd.to_datetime(s, dayfirst=True, errors='coerce')

all_data['date_parsed'] = all_data['date'].apply(try_parse_date)
ok = all_data['date_parsed'].notna().sum()
bad = len(all_data) - ok
print(f"日期解析成功：{ok:,} 行；失败：{bad:,} 行")

# 仅对成功解析的行做 2015–2025 过滤；解析失败的行保留（方便后续人工修正）
mask_ok = all_data['date_parsed'].between('2015-01-01','2025-12-31', inclusive='both')
mask_bad = all_data['date_parsed'].isna()
final = all_data[mask_ok | mask_bad].copy()
print(f"筛选后（保留无效日期）：{len(final):,} 行")

final.to_csv(OUT, index=False)
print(f"已保存：{OUT}    用时 {time.time()-start:.2f}s")


共找到 1262 个 CSV 文件



读取 CSV: 100%|██████████| 1262/1262 [09:51<00:00,  2.13文件/s]


 读取报告已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report.csv
—— Top 5 读取最少行的文件 ——
                                                   file  rows used_sep  \
1240  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1241  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1242  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1243  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1244  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   

     used_encoding error  
1240         utf-8        
1241         utf-8        
1242         utf-8        
1243         utf-8        
1244         utf-8        
合并后总行数：1,262
日期解析成功：899 行；失败：363 行
筛选后（保留无效日期）：1,262 行
已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_all_data_2015_2025.csv    用时 597.02s


In [ ]:
#改正过后的代码


# 递归合并（带分隔符/编码自适应）+ 强化日期解析 + 基础去重 + 详尽报告
import pandas as pd
import numpy as np
import os, time, re
from tqdm import tqdm

start = time.time()

# === 路径设置（与原脚本一致） ===
FOLDER = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/ch_corpus'
OUT    = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_all_data_2015_2025.csv'
REPORT = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report.csv'
REPORT_STATS = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report_stats.csv'

REQUIRED_COLS = ['author','title','date','content','url']  # 你的目标列

# === 读取函数：更多编码与分隔符 + 自动嗅探 ===
def read_csv_robust(path):
    """返回 (df, used_sep, used_enc, err)"""
    encs_try = ['utf-8', 'utf-8-sig', 'gb18030', 'gbk', 'big5', 'cp1252', 'latin1']
    seps_try = [None, ',', '\t', ';', '|']   # None=limited sniffing
    last_err = None
    for enc in encs_try:
        for sep in seps_try:
            try:
                df = pd.read_csv(
                    path,
                    sep=sep,
                    engine='python',
                    encoding=enc,
                    on_bad_lines='skip'
                )
                if df is None:
                    continue
                return df, (sep if sep is not None else 'auto'), enc, None
            except Exception as e:
                last_err = str(e)
                continue
    return pd.DataFrame(), 'fail', 'fail', last_err

# === 日期解析增强：中文/时间戳/ISO/RFC 3339/常见格式 ===
_ts10 = re.compile(r'^\d{10}$')   # 秒
_ts13 = re.compile(r'^\d{13}$')   # 毫秒
_just_digits = re.compile(r'^\d+$')

def normalize_cn_date(s: str) -> str:
    """把中文日期部件替换成统一符号；去掉中文‘日’/‘号’/星期等；统一标点"""
    s = s.strip()
    # 去中文星期
    s = re.sub(r'星期[一二三四五六日天]|周[一二三四五六日天]', '', s)
    # 年月日替换
    s = s.replace('年','-').replace('月','-').replace('日','').replace('号','')
    # 全角与杂字符
    s = s.replace('\u3000',' ').replace('：',':').replace('．','.')
    s = re.sub(r'\s+', ' ', s)
    return s

def try_parse_date(val):
    if pd.isna(val):
        return pd.NaT
    s = str(val).strip()
    if not s:
        return pd.NaT

    # 纯数字时间戳
    if _just_digits.match(s):
        try:
            if _ts13.match(s):  # 毫秒
                return pd.to_datetime(int(s), unit='ms', utc=False)
            if _ts10.match(s):  # 秒
                return pd.to_datetime(int(s), unit='s', utc=False)
        except:
            pass

    # 规范化中文日期
    s_norm = normalize_cn_date(s)

    # 先按常见格式试
    fmts = [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %H:%M",
        "%Y-%m-%d",
        "%Y/%m/%d %H:%M:%S",
        "%Y/%m/%d %H:%M",
        "%Y/%m/%d",
        "%Y.%m.%d %H:%M:%S",
        "%Y.%m.%d",
        "%d/%m/%Y %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y",
    ]
    for fmt in fmts:
        try:
            return pd.to_datetime(s_norm, format=fmt)
        except:
            pass

    # 最后宽松解析（含 ISO8601/RFC 3339、带时区等；dayfirst=True）
    try:
        return pd.to_datetime(s_norm, errors='coerce', utc=False, infer_datetime_format=False, dayfirst=True)
    except:
        return pd.NaT

# === 基础工具 ===
def detect_source_from_path(fp: str) -> str:
    # 你之前的规则保留，同时从路径抽取一个可读的上级目录作为回退
    for s in ['Kenh','Genk','Cafebize']:
        if re.search(rf'/{s}/', fp, flags=re.I):
            return s.lower()
    # 回退：取倒数第2或第1级目录名做 source
    parts = [p for p in fp.split(os.sep) if p]
    if len(parts) >= 2:
        return parts[-2].lower()
    return 'unknown'

def normalize_url(u):
    if pd.isna(u):
        return u
    u = str(u).strip()
    if not u:
        return u
    # 简单归一化：小写 + 去掉末尾斜杠
    u = u.lower()
    u = re.sub(r'//+', '/', u)  # 多斜杠
    u = re.sub(r'^https:/','https://',u)
    u = re.sub(r'^http:/','http://',u)
    u = u.rstrip('/')
    return u

# === 主流程 ===
if not os.path.exists(FOLDER):
    raise FileNotFoundError(f"路径不存在: {FOLDER}")

# 收集 CSV
csv_files = []
for root, _, files in os.walk(FOLDER):
    for f in files:
        if f.lower().endswith('.csv'):
            csv_files.append(os.path.join(root, f))
print(f"共找到 {len(csv_files)} 个 CSV 文件\n")

# 读取 + 报告
records = []
dfs = []

for fp in tqdm(csv_files, desc="读取 CSV", unit="文件"):
    df, used_sep, used_enc, err = read_csv_robust(fp)

    # 标注来源、路径年份提示
    source = detect_source_from_path(fp)
    y_hint = None
    y = re.search(r'(20[1-2]\d)', fp)
    if y:
        y_hint = y.group(1)

    # 统一列名（去空格+原样；再映射到小写）
    old_cols = df.columns.tolist()
    df.columns = [c.strip() for c in df.columns]
    lower_map = {c: c.lower() for c in df.columns}
    df.rename(columns=lower_map, inplace=True)

    row_count = len(df)
    has_cols = {c: (c in df.columns) for c in REQUIRED_COLS}

    records.append({
        'file': fp,
        'rows': row_count,
        'used_sep': used_sep,
        'used_encoding': used_enc,
        'has_author': has_cols['author'],
        'has_title': has_cols['title'],
        'has_date': has_cols['date'],
        'has_content': has_cols['content'],
        'has_url': has_cols['url'],
        'source': source,
        'year_hint': y_hint,
        'error': '' if err is None else err[:200]
    })

    # 只要不是“完全空表”就收；缺列补 None，确保结构统一
    if row_count > 0:
        for c in REQUIRED_COLS:
            if c not in df.columns:
                df[c] = None
        df['source'] = source
        df['year_hint'] = y_hint
        # 保留基础列
        dfs.append(df[REQUIRED_COLS + ['source','year_hint']])

# 输出读取报告（逐文件）
report_df = pd.DataFrame.from_records(records)
report_df.to_csv(REPORT, index=False, encoding='utf-8')
print(f" 读取报告已保存：{REPORT}")

print("—— Top 5 读取最少行的文件 ——")
if not report_df.empty:
    print(report_df.sort_values('rows').head(5)[['file','rows','used_sep','used_encoding','error']])

if not dfs:
    raise ValueError("所有文件都读空了，请检查读取报告（merge_report.csv）")

# 合并
all_data = pd.concat(dfs, ignore_index=True)
print(f"合并后总行数：{len(all_data):,}")

# 轻度清洗：去首尾空白
for c in REQUIRED_COLS:
    all_data[c] = all_data[c].astype(str).str.strip().replace({'': np.nan})

# 解析日期
all_data['date_parsed'] = all_data['date'].apply(try_parse_date)
ok = all_data['date_parsed'].notna().sum()
bad = len(all_data) - ok
print(f"日期解析成功：{ok:,} 行；失败：{bad:,} 行")

# 可选：对解析失败但有 year_hint 的行做“软补年”，便于年度统计（可追溯）
mask_bad = all_data['date_parsed'].isna()
has_yhint = all_data['year_hint'].notna()
fill_mask = mask_bad & has_yhint
all_data['date_filled'] = all_data['date_parsed']
all_data.loc[fill_mask, 'date_filled'] = pd.to_datetime(
    all_data.loc[fill_mask, 'year_hint'].astype(str) + '-01-01', errors='coerce'
)

# 仅对成功解析的行做 2015–2025 过滤；解析失败的行保留（与你原策略一致）
mask_ok = all_data['date_parsed'].between('2015-01-01','2025-12-31', inclusive='both')
final = all_data[mask_ok | mask_bad].copy()
print(f"筛选后（保留无效日期）：{len(final):,} 行")

# ===== 基础去重 =====
# 1) URL 归一化后去重（优先）
final['url_norm'] = final['url'].apply(normalize_url)
before = len(final)
final = final.drop_duplicates(subset=['url_norm'], keep='first')
after = len(final)
print(f"按 URL 去重：移除 {before - after:,} 行，剩余 {after:,} 行")

# 2) 兜底：对有 title 且有 date_filled 的再按 (title, date_filled) 去重
mask_title = final['title'].notna() & (final['title'].str.len() > 0)
mask_dtok = final['date_filled'].notna()
before = len(final)
final = final.sort_values(by=['date_filled','title']).drop_duplicates(subset=['title','date_filled'], keep='first')
after2 = len(final)
print(f"按 (title, date_filled) 兜底去重：再移除 {before - after2:,} 行，剩余 {after2:,} 行")

# 导出主结果
# 列顺序：核心列 + 解析列 + 归一化/辅助列
cols_out = REQUIRED_COLS + ['source','year_hint','date_parsed','date_filled','url_norm']
# 若某列不存在（极端情况），跳过
cols_out = [c for c in cols_out if c in final.columns]
final.to_csv(OUT, index=False, encoding='utf-8')
print(f"已保存：{OUT}    用时 {time.time()-start:.2f}s")

# ===== 统计报告（更可读的质量视图）=====
stats = {}

# 1) 总览
stats['total_files'] = [len(csv_files)]
stats['total_rows_after_concat'] = [len(all_data)]
stats['date_parse_ok'] = [ok]
stats['date_parse_bad'] = [bad]
stats['rows_after_filter_keep_bad'] = [len(all_data[mask_ok | mask_bad])]
stats['rows_after_dedup'] = [len(final)]

# 2) 编码/分隔符分布（基于逐文件报告）
enc_dist = (report_df['used_encoding']
            .value_counts(dropna=False)
            .rename_axis('encoding').reset_index(name='files'))
sep_dist = (report_df['used_sep']
            .value_counts(dropna=False)
            .rename_axis('sep').reset_index(name='files'))

# 3) 来源的日期命中率（基于逐行）
src_ok = final.assign(_ok=final['date_parsed'].notna()).groupby('source')['_ok'].agg(['sum','count'])
src_ok['ok_rate'] = (src_ok['sum'] / src_ok['count']).round(4)
src_ok = src_ok.reset_index().rename(columns={'sum':'ok_rows','count':'total_rows'})

# 4) 年度分布（优先用 date_parsed，其次用 date_filled）
year_series = pd.Series(pd.NaT, index=final.index)
year_series.loc[final['date_parsed'].notna()] = final.loc[final['date_parsed'].notna(), 'date_parsed']
year_series.loc[final['date_parsed'].isna() & final['date_filled'].notna()] = final.loc[final['date_parsed'].isna() & final['date_filled'].notna(), 'date_filled']
year = pd.to_datetime(year_series, errors='coerce').dt.year
year_dist = year.value_counts(dropna=False).sort_index().rename_axis('year').reset_index(name='rows')

# 5) 缺列概览（按文件）
missing_cols = []
for c in REQUIRED_COLS:
    missing_cols.append(
        report_df[~report_df[f'has_{c}']].shape[0]
    )
missing_df = pd.DataFrame({
    'col': REQUIRED_COLS,
    'files_missing': missing_cols
})

# 6) 写出 stats 到 CSV（多表合并为一个文件，添加 tag 区分）
def tag_df(df, tag):
    out = df.copy()
    out.insert(0, '_section', tag)
    return out

overview_df = pd.DataFrame(stats)
overview_df = tag_df(overview_df, 'overview')

enc_dist = tag_df(enc_dist, 'encoding_dist')
sep_dist = tag_df(sep_dist, 'sep_dist')
src_ok = tag_df(src_ok, 'source_date_ok')
year_dist = tag_df(year_dist, 'year_dist')
missing_df = tag_df(missing_df, 'missing_required_cols')

stats_out = pd.concat([overview_df, enc_dist, sep_dist, src_ok, year_dist, missing_df], ignore_index=True)
stats_out.to_csv(REPORT_STATS, index=False, encoding='utf-8')
print(f"统计报告已保存：{REPORT_STATS}")

# 屏幕上再打印些关键信息
print("\n—— 编码分布（前10）——")
print(enc_dist.head(10))
print("\n—— 分隔符分布（前10）——")
print(sep_dist.head(10))
print("\n—— 各来源日期命中率（前10）——")
print(src_ok.sort_values('ok_rate', ascending=False).head(10))
print("\n—— 年度分布 ——")
print(year_dist)
print("\n—— 必需列缺失（按文件计数）——")
print(missing_df)


共找到 1286 个 CSV 文件



读取 CSV: 100%|██████████| 1286/1286 [00:06<00:00, 191.53文件/s]


 读取报告已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report.csv
—— Top 5 读取最少行的文件 ——
                                                   file  rows used_sep  \
1272  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1273  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1274  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1275  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
1276  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   

     used_encoding error  
1272         utf-8        
1273         utf-8        
1274         utf-8        
1275         utf-8        
1276         utf-8        
合并后总行数：1,286


/tmp/ipython-input-1557293574.py:106: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s_norm, errors='coerce', utc=False, infer_datetime_format=False, dayfirst=True)


日期解析成功：945 行；失败：341 行
筛选后（保留无效日期）：1,286 行
按 URL 去重：移除 775 行，剩余 511 行
按 (title, date_filled) 兜底去重：再移除 4 行，剩余 507 行
已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_all_data_2015_2025.csv    用时 7.42s
统计报告已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_merge_report_stats.csv

—— 编码分布（前10）——
        _section encoding  files
0  encoding_dist    utf-8   1286

—— 分隔符分布（前10）——
   _section   sep  files
0  sep_dist  auto   1286

—— 各来源日期命中率（前10）——
          _section source  ok_rows  total_rows  ok_rate
9   source_date_ok   2024       35          35   1.0000
10  source_date_ok   2025       25          25   1.0000
4   source_date_ok   2019       41          42   0.9762
5   source_date_ok   2020       33          39   0.8462
8   source_date_ok   2023       27          36   0.7500
7   source_date_ok   2022       24          43   0.5581
6   source_date_ok   2021       21          39   0.5385
3   source_date_ok   

In [ ]:
#第二次更改版本


import pandas as pd
import os, time, re
from tqdm import tqdm

start = time.time()

FOLDER = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/ch_corpus'
OUT    = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_all_data_2015_2025.csv'
REPORT = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_merge_report.csv'

REQUIRED_COLS = ['author','title','date','content','url']

def read_csv_robust(path):
    seps_try = [None, ',', ';', '\t', '|']
    encs_try = ['utf-8', 'utf-8-sig', 'latin1']
    for enc in encs_try:
        for sep in seps_try:
            try:
                df = pd.read_csv(path, sep=sep, engine='python', encoding=enc, on_bad_lines='skip')
                return df, (sep if sep else 'auto'), enc, None
            except Exception as e:
                last_err = str(e)
    return pd.DataFrame(), 'fail', 'fail', last_err

if not os.path.exists(FOLDER):
    raise FileNotFoundError(f"路径不存在: {FOLDER}")

csv_files = [os.path.join(root, f) for root, _, files in os.walk(FOLDER) for f in files if f.lower().endswith('.csv')]
print(f"共找到 {len(csv_files)} 个 CSV 文件\n")

records, dfs = [], []

for fp in tqdm(csv_files, desc="读取 CSV", unit="文件"):
    df, used_sep, used_enc, err = read_csv_robust(fp)

    # 去掉不可见字符（防止 BOM / 全角空格）
    df.columns = [c.strip().replace('\ufeff','') for c in df.columns]
    lower_map = {c: c.lower() for c in df.columns}
    df.rename(columns=lower_map, inplace=True)

    row_count = len(df)
    has_cols = {c: (c in df.columns) for c in REQUIRED_COLS}

    records.append({
        'file': fp, 'rows': row_count,
        'used_sep': used_sep, 'used_encoding': used_enc,
        **{f'has_{c}': has_cols[c] for c in REQUIRED_COLS},
        'error': '' if err is None else err[:120]
    })

    if row_count > 0:
        for c in REQUIRED_COLS:
            if c not in df.columns:
                df[c] = None
        dfs.append(df[REQUIRED_COLS])

report_df = pd.DataFrame.from_records(records)
report_df.to_csv(REPORT, index=False)
print(f"读取报告已保存：{REPORT}")

if not dfs:
    raise ValueError("所有文件都读空了，请检查 merge_report.csv")

all_data = pd.concat(dfs, ignore_index=True)

# 日期解析
def try_parse_date(s):
    if pd.isna(s): return pd.NaT
    s = str(s).strip()
    for fmt in ("%d/%m/%Y %I:%M %p","%d/%m/%Y %H:%M","%d/%m/%Y",
                "%Y-%m-%d %H:%M:%S","%Y-%m-%d %H:%M","%Y-%m-%d",
                "%Y/%m/%d %H:%M:%S","%Y/%m/%d"):
        try:
            return pd.to_datetime(s, format=fmt)
        except:
            pass
    return pd.to_datetime(s, dayfirst=True, errors='coerce')

all_data['date'] = pd.to_datetime(all_data['date'], errors='coerce')

mask_ok = all_data['date'].between('2015-01-01','2025-12-31', inclusive='both')
mask_bad = all_data['date'].isna()
final = all_data[mask_ok | mask_bad].copy()

final.to_csv(OUT, index=False)
print(f"已保存：{OUT}    用时 {time.time()-start:.2f}s")


共找到 1286 个 CSV 文件



读取 CSV: 100%|██████████| 1286/1286 [00:17<00:00, 72.02文件/s] 
/tmp/ipython-input-1832548863.py:82: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['date'] = pd.to_datetime(all_data['date'], errors='coerce')


读取报告已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_merge_report.csv
已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_all_data_2015_2025.csv    用时 18.42s


#Tokenisation en chinois



In [ ]:
#Thulac

!pip -q install thulac tqdm pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 MB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import thulac
from tqdm import tqdm

# 1) 初始化（seg_only=True 只分词不标注词性，快）
lac = thulac.thulac(seg_only=True)  # 第一次调用会加载模型

# 2) 路径与读取
IN_CSV  = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_all_data_2015_2025.csv'
OUT_CSV = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch1_all_data_2015_2025_tok_thulac.csv'

df = pd.read_csv(IN_CSV, encoding='utf-8')
if 'content' not in df.columns:
    raise ValueError("CSV 里没有 'content' 列，请确认。")

# 3) 分词函数
def zh_tokenize_thulac(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text).strip()
    if not s:
        return ""
    # lac.cut(s, text=True) 直接返回分好词的字符串，默认用空格分开
    return lac.cut(s, text=True)

# 4) 批处理
tqdm.pandas(desc="THULAC 分词")
df['content_tok'] = df['content'].progress_apply(zh_tokenize_thulac)

# 5) 导出
df.to_csv(OUT_CSV, index=False, encoding='utf-8')
print(f"已保存：{OUT_CSV}\n预览：")
print(df[['author','title','date','content_tok']].head(3))

Model loaded succeed


THULAC 分词: 100%|██████████| 1282/1282 [00:41<00:00, 31.23it/s]


已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch1_all_data_2015_2025_tok_thulac.csv
预览：
     author title date                                        content_tok
0  (王连香、高雷)   NaN  NaN  人民网 北京 9月 28日 电 （ 记者 王 连 香 ） 在 今日 举行 的 交通 运输部 ...
1  (王连香、高雷)   NaN  NaN  人民网 北京 9月 28日 电 （ 记者 王 连 香 ） 据 中国 国 家 铁路 集团 有限...
2  (袁勃、胡永秋)   NaN  NaN  新华社 北京 9月 28日 电 （ 记者 冯歆然 ） 外交部 发言人 汪文斌 28日 表示 ...


In [ ]:
import pandas as pd
import thulac
from tqdm import tqdm

# 1) 初始化（seg_only=True 只分词不标注词性，快）
lac = thulac.thulac(seg_only=True)  # 第一次调用会加载模型

# 2) 路径与读取
IN_CSV  = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch1_all_data_2015_2025.csv'
OUT_CSV = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch2_all_data_2015_2025_tok_thulac.csv'

df = pd.read_csv(IN_CSV, encoding='utf-8')
if 'content' not in df.columns:
    raise ValueError("CSV 里没有 'content' 列，请确认。")

# 3) 分词函数
def zh_tokenize_thulac(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text).strip()
    if not s:
        return ""
    return lac.cut(s, text=True)  # 直接输出空格分词

# 4) 批处理
tqdm.pandas(desc="THULAC 分词")
df['content'] = df['content'].progress_apply(zh_tokenize_thulac)

# 5) 只保留需要的列（去掉原始未分词文本）
cols_to_keep = [c for c in df.columns if c != 'content_tok']  # 没有content_tok了
df = df[['author','title','date','content']]  # 保留分词后content

# 6) 导出
df.to_csv(OUT_CSV, index=False, encoding='utf-8')
print(f"已保存：{OUT_CSV}\n预览：")
print(df.head(3))


Model loaded succeed


THULAC 分词: 100%|██████████| 1282/1282 [00:37<00:00, 33.80it/s]


已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch2_all_data_2015_2025_tok_thulac.csv
预览：
     author title date                                            content
0  (王连香、高雷)   NaN  NaN  人民网 北京 9月 28日 电 （ 记者 王 连 香 ） 在 今日 举行 的 交通 运输部 ...
1  (王连香、高雷)   NaN  NaN  人民网 北京 9月 28日 电 （ 记者 王 连 香 ） 据 中国 国 家 铁路 集团 有限...
2  (袁勃、胡永秋)   NaN  NaN  新华社 北京 9月 28日 电 （ 记者 冯歆然 ） 外交部 发言人 汪文斌 28日 表示 ...


In [ ]:
#Stanza

!pip -q install stanza tqdm pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 959.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 安装与初始化
!pip -q install stanza tqdm pandas

import pandas as pd
import stanza
from tqdm import tqdm

# 1) 下载中文模型（只需首次）
stanza.download('zh')  # 如果已下载过会跳过

# 2) 初始化中文流水线（只开 tokenize，速度更快）
nlp = stanza.Pipeline(lang='zh', processors='tokenize', tokenize_pretokenized=False, use_gpu=False)

# 3) 路径与读取
IN_CSV  = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/ch_all_data_2015_2025.csv'
OUT_CSV = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch_all_data_2015_2025_tok_stanza.csv'

df = pd.read_csv(IN_CSV, encoding='utf-8')
if 'content' not in df.columns:
    raise ValueError("CSV 里没有 'content' 列，请确认。")

# 4) 分词函数（对 NaN 做保护）
def zh_tokenize_stanza(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text).strip()
    if not s:
        return ""
    doc = nlp(s)
    tokens = []
    for sent in doc.sentences:
        for w in sent.tokens:
            tokens.append(w.text)
    return " ".join(tokens)

# 5) 直接替换原来的 content 列
tqdm.pandas(desc="Stanza 分词")
df['content'] = df['content'].progress_apply(zh_tokenize_stanza)

# 6) 导出
df.to_csv(OUT_CSV, index=False, encoding='utf-8')
print(f"已保存：{OUT_CSV}\n预览：")
print(df[['author','title','date','content']].head(3))


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...
INFO:stanza:File exists: /root/stanza_resources/zh-hans/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!
Stanza 分词: 100%|██████████| 507/507 [00:40<00:00, 12.49it/s]

已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/ch_all_data_2015_2025_tok_stanza.csv
预览：
   author                     title              date  \
0     NaN    2015人民微博影响力榜单发布 五大榜单出炉  2015年12月31日16:18   
1     NaN    2015人民微博影響力榜單發布 五大榜單出爐  2015年12月31日16:18   
2     NaN  2015优秀网络视听作品推选  人民网获两项大奖  2015年12月14日16:12   

                                             content  
0  人民 网 12 月 31 日 电 又是 岁 末年 初 ， 回想 走过 的 这 一 年 ， 人...  
1  人民 網 12 月 31 日 電 又是 歲 末年 初 ， 回想 走過 的 這 一 年 ， 人...  
2  近日 ， 在 “ 2015 优秀 网络 视听 作品 推选 活动 ” 中 ， 人民 网 品牌 ...  


In [ ]:
import re

input_file = "input.txt"
output_file = "output.txt"

with open(input_file, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
    for line in f_in:
        # 只匹配以 Bản mẫu 开头的 text
        m = re.search(r'"text":\s*"Bản mẫu:([^"]+)"', line)
        if m:
            content = m.group(1)
            # 如果有 ":-" 或 ":*"，取前半部分
            if ":-" in content:
                content = content.split(":-", 1)[0]
            if ":*" in content:
                content = content.split(":*", 1)[0]
            f_out.write(content.strip() + "\n")

print("处理完成，结果保存在", output_file)


#2. Référence wiktionary en vietnamien

words in 2018  

In [ ]:
import json
import re

input_file = "/content/words2018_2.txt"       # 你的源文件
output_file = "/content/words2018_2_clean.txt"  # 输出结果

def is_vietnamese_word(s: str) -> bool:
    s = s.strip()
    # 排除空字符串
    if not s:
        return False
    # 排除只有一个字符的条目
    if len(s) == 1:
        return False
    # 排除只含一个带声调字母的条目
    if len(s.replace(" ", "")) == 1:
        return False
    # 允许字母、空格、连字符和撇号
    if not re.match(r"^[\wÀ-ỹà-ỹ\s'\-]+$", s):
        return False
    return True

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            continue
        text = obj.get("text", "").strip()
        if is_vietnamese_word(text):
            fout.write(text + "\n")

print("处理完成！结果已保存到：", output_file)


wiktionary

In [ ]:
# Colab-ready: 抓取“2015 年以前”的越南 Wiktionary 词条标题为词表（JSONL）
import requests, time, json, math
from datetime import datetime, timezone

API = "https://vi.wiktionary.org/w/api.php"
CUTOFF = datetime(2014, 12, 31, 23, 59, 59, tzinfo=timezone.utc)  # 截止到 2014-12-31
OUT = "wiktionary_vi_pre2015.jsonl"

session = requests.Session()
session.headers.update({
    "User-Agent": "VN-Wiktionary-Pre2015-Extractor/1.0 (research; contact: youremail@example.com)"
})

def wiktionary_all_titles():
    """遍历越南 Wiktionary 主命名空间（0）的所有非重定向页面，yield 标题"""
    apcontinue = None
    while True:
        params = {
            "action": "query",
            "format": "json",
            "list": "allpages",
            "apnamespace": "0",          # 主命名空间
            "apfilterredir": "nonredirects",
            "aplimit": "500",            # 最大 500
        }
        if apcontinue:
            params["apcontinue"] = apcontinue

        r = session.get(API, params=params, timeout=60)
        r.raise_for_status()
        data = r.json()
        for p in data.get("query", {}).get("allpages", []):
            title = p.get("title", "").strip()
            if title and (":" not in title):  # 保险起见，再排除带命名空间前缀的
                yield title

        if "continue" in data and "apcontinue" in data["continue"]:
            apcontinue = data["continue"]["apcontinue"]
            # 轻微限速，避免触发流控
            time.sleep(0.15)
        else:
            break

def first_revision_timestamp(titles):
    """
    批量查询每个标题的首个修订时间。titles: list[str] (len<=50)
    返回 dict[title] = datetime (UTC) 或 None（若查不到）
    """
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "rvprop": "timestamp",
        "rvlimit": "1",
        "rvdir": "newer",
        "formatversion": "2",
        "titles": "|".join(titles)
    }
    r = session.get(API, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()
    out = {}
    for page in data.get("query", {}).get("pages", []):
        title = page.get("title")
        revs = page.get("revisions")
        if title and revs:
            ts = revs[0]["timestamp"]  # e.g. "2009-01-02T03:04:05Z"
            out[title] = datetime.fromisoformat(ts.replace("Z", "+00:00"))
        else:
            out[title] = None
    return out

def batched(iterable, n=50):
    buf = []
    for x in iterable:
        buf.append(x)
        if len(buf) >= n:
            yield buf
            buf = []
    if buf:
        yield buf

kept = 0
seen = 0
with open(OUT, "w", encoding="utf-8") as fout:
    for batch in batched(wiktionary_all_titles(), 50):
        # 查询该批次的首修时间
        for _ in range(3):  # 简单重试
            try:
                ts_map = first_revision_timestamp(batch)
                break
            except requests.RequestException:
                time.sleep(1.0)
        else:
            # 连续失败，跳过该批
            ts_map = {t: None for t in batch}

        for title, ts in ts_map.items():
            seen += 1
            if ts is not None and ts <= CUTOFF:
                obj = {"text": title, "source": "wiktionary"}
                fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
                kept += 1

        # 温和限速
        time.sleep(0.2)
        if seen % 500 == 0:
            print(f"processed={seen}, kept(pre-2015)={kept}")

print(f"完成！共处理 {seen} 个词条，保留（≤2014-12-31）{kept} 个。输出文件：{OUT}")


processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=0, kept(pre-2015)=0
processed=

#Tokenisation en vietnamien

In [ ]:
#步骤 1：将所有 CSV 文件合并成一个文件

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#2. 递归合并所有 CSV（2015–2025）计时
import pandas as pd
import os
import time
from tqdm import tqdm  # 用于显示进度条

start_time = time.time()

# 主目录
folder_path = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/vi-corpus'

if not os.path.exists(folder_path):
    raise FileNotFoundError(f"路径不存在: {folder_path}")

# 遍历所有子文件夹，收集 csv 文件路径
csv_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith('.csv'):
            csv_files.append(os.path.join(root, file))

print(f"共找到 {len(csv_files)} 个 CSV 文件\n")

# 合并（带进度条）
df_list = []
for file_path in tqdm(csv_files, desc="正在读取 CSV 文件", unit="文件"):
    try:
        df = pd.read_csv(file_path)
        df_list.append(df)
    except Exception as e:
        print(f" 读取失败 {file_path}: {e}")

if not df_list:
    raise ValueError("没有找到可合并的 CSV 数据")

# 合并所有 DataFrame
all_data = pd.concat(df_list, ignore_index=True)

# 如果 date 列可用，过滤 2015–2025
if 'date' in all_data.columns:
    all_data['date'] = pd.to_datetime(all_data['date'], errors='coerce')
    all_data = all_data[
        (all_data['date'].dt.year >= 2015) &
        (all_data['date'].dt.year <= 2025)
    ]

# 保存结果
merged_path = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_all_data_2015_2025.csv'
all_data.to_csv(merged_path, index=False)

# 打印总用时
elapsed_time = time.time() - start_time
print(f"\n合并完成，保存到: {merged_path}")
print(f"总用时: {elapsed_time:.2f} 秒")


共找到 759 个 CSV 文件



正在读取 CSV 文件: 100%|██████████| 759/759 [00:02<00:00, 280.49文件/s]



合并完成，保存到: /content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_all_data_2015_2025.csv
总用时: 3.09 秒


/tmp/ipython-input-2970731906.py:41: UserWarning: Parsing dates in %d/%m/%Y %I:%M %p format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  all_data['date'] = pd.to_datetime(all_data['date'], errors='coerce')


In [ ]:
# 递归合并（带分隔符/编码自适应）+ 详尽报告
import pandas as pd
import os, time, re
from tqdm import tqdm

start = time.time()

FOLDER = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/vi-corpus'
OUT    = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/vi_all_data_2015_2025.csv'
REPORT = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/vi_merge_report.csv'

REQUIRED_COLS = ['author','title','date','content','url']  # 你的目标列

def read_csv_robust(path):
    """返回 (df, used_sep, used_enc, err)"""
    seps_try = [None, ',', ';', '\t', '|']   # None=自动嗅探（engine='python'）
    encs_try = ['utf-8', 'utf-8-sig', 'latin1']
    last_err = None
    for enc in encs_try:
        for sep in seps_try:
            try:
                df = pd.read_csv(
                    path,
                    sep=sep,
                    engine='python',
                    encoding=enc,
                    on_bad_lines='skip'
                )
                return df, (sep if sep is not None else 'auto'), enc, None
            except Exception as e:
                last_err = str(e)
                continue
    return pd.DataFrame(), 'fail', 'fail', last_err

if not os.path.exists(FOLDER):
    raise FileNotFoundError(f"路径不存在: {FOLDER}")

# 收集 CSV
csv_files = []
for root, _, files in os.walk(FOLDER):
    for f in files:
        if f.lower().endswith('.csv'):
            csv_files.append(os.path.join(root, f))
print(f"共找到 {len(csv_files)} 个 CSV 文件\n")

# 读取 + 报告
records = []
dfs = []

for fp in tqdm(csv_files, desc="读取 CSV", unit="文件"):
    df, used_sep, used_enc, err = read_csv_robust(fp)

    # 标注来源、路径年份提示
    source = 'unknown'
    for s in ['Kenh','Genk','Cafebize']:
        if re.search(rf'/{s}/', fp, flags=re.I):
            source = s.lower()
            break
    y_hint = None
    y = re.search(r'(20[1-2]\d)', fp)
    if y: y_hint = y.group(1)

    # 统一列名（去空格小写），避免“Date”“URL”之类大小写不一致
    old_cols = df.columns.tolist()
    df.columns = [c.strip() for c in df.columns]
    lower_map = {c: c.lower() for c in df.columns}
    df.rename(columns=lower_map, inplace=True)

    row_count = len(df)
    has_cols = {c: (c in df.columns) for c in REQUIRED_COLS}

    records.append({
        'file': fp,
        'rows': row_count,
        'used_sep': used_sep,
        'used_encoding': used_enc,
        'has_author': has_cols['author'],
        'has_title': has_cols['title'],
        'has_date': has_cols['date'],
        'has_content': has_cols['content'],
        'has_url': has_cols['url'],
        'source': source,
        'year_hint': y_hint,
        'error': '' if err is None else err[:120]
    })

    # 只要不是“完全空表”就先收进来；后续再统一处理日期
    if row_count > 0:
        # 缺的列补空，确保结构统一
        for c in REQUIRED_COLS:
            if c not in df.columns:
                df[c] = None
        df['source'] = source
        df['year_hint'] = y_hint
        dfs.append(df[REQUIRED_COLS + ['source','year_hint']])

# 输出报告
report_df = pd.DataFrame.from_records(records)
report_df.to_csv(REPORT, index=False)
print(f" 读取报告已保存：{REPORT}")
print("—— Top 5 读取最少行的文件 ——")
print(report_df.sort_values('rows').head(5)[['file','rows','used_sep','used_encoding','error']])

if not dfs:
    raise ValueError("所有文件都读空了，请检查报告 merge_report.csv")

# 合并
all_data = pd.concat(dfs, ignore_index=True)
print(f"合并后总行数：{len(all_data):,}")

# 日期解析（尽量保留；只对成功解析的行做年份过滤，失败的行也保留）
def try_parse_date(s):
    if pd.isna(s): return pd.NaT
    s = str(s).strip()
    for fmt in ("%d/%m/%Y %I:%M %p","%d/%m/%Y %H:%M","%d/%m/%Y",
                "%Y-%m-%d %H:%M:%S","%Y-%m-%d %H:%M","%Y-%m-%d",
                "%Y/%m/%d %H:%M:%S","%Y/%m/%d"):
        try: return pd.to_datetime(s, format=fmt)
        except: pass
    return pd.to_datetime(s, dayfirst=True, errors='coerce')

all_data['date_parsed'] = all_data['date'].apply(try_parse_date)
ok = all_data['date_parsed'].notna().sum()
bad = len(all_data) - ok
print(f"日期解析成功：{ok:,} 行；失败：{bad:,} 行")

# 仅对成功解析的行做 2015–2025 过滤；解析失败的行保留（方便后续人工修正）
mask_ok = all_data['date_parsed'].between('2015-01-01','2025-12-31', inclusive='both')
mask_bad = all_data['date_parsed'].isna()
final = all_data[mask_ok | mask_bad].copy()
print(f"筛选后（保留无效日期）：{len(final):,} 行")

final.to_csv(OUT, index=False)
print(f"已保存：{OUT}    用时 {time.time()-start:.2f}s")


共找到 759 个 CSV 文件



读取 CSV: 100%|██████████| 759/759 [00:03<00:00, 194.61文件/s]
/tmp/ipython-input-56446194.py:120: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors='coerce')


 读取报告已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_merge_report.csv
—— Top 5 读取最少行的文件 ——
                                                  file  rows used_sep  \
751  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
750  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
749  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
748  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   
747  /content/drive/MyDrive/Colab Notebooks/STAGE_C...     1     auto   

    used_encoding error  
751         utf-8        
750         utf-8        
749         utf-8        
748         utf-8        
747         utf-8        
合并后总行数：759


/tmp/ipython-input-56446194.py:120: UserWarning: Parsing dates in %m:%M %d/%H/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors='coerce')
/tmp/ipython-input-56446194.py:120: UserWarning: Parsing dates in %H:%m %d/%M/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors='coerce')


日期解析成功：627 行；失败：132 行
筛选后（保留无效日期）：758 行
已保存：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_all_data_2015_2025.csv    用时 4.67s


In [ ]:
#先按年份合并，再把每年的结果再合并成一个总表，代码结构要稍微改，比如这样：

import pandas as pd
import os

folder_path = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Avant-Token/vi-corpus'

if not os.path.exists(folder_path):
    raise FileNotFoundError(f"路径不存在: {folder_path}")

# 保存所有年份的结果
yearly_dfs = []

for year in range(2015, 2026):  # 2015 到 2025
    year_csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.csv') and str(year) in file:
                year_csv_files.append(os.path.join(root, file))

    if year_csv_files:
        df_list = []
        for file_path in year_csv_files:
            try:
                df = pd.read_csv(file_path)
                df_list.append(df)
            except Exception as e:
                print(f"{year} 读取失败 {file_path}: {e}")

        if df_list:
            year_df = pd.concat(df_list, ignore_index=True)
            yearly_dfs.append(year_df)

# 再把所有年份的数据合并成一个总表
if yearly_dfs:
    all_data = pd.concat(yearly_dfs, ignore_index=True)
    merged_path = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/all_data_2015_2025.csv'
    all_data.to_csv(merged_path, index=False)
    print(f"按年份合并完成，保存到: {merged_path}")
else:
    print("没有找到任何数据")


Token

In [ ]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00


In [ ]:
#3. 用 PyVi 分词越南语

# 用 PyVi 对 content 列分词（大文件友好、带进度条）
import os, time, re
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer

INPUT  = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/fusion_avantToken/vi_all_data_2015_2025.csv'
OUTPUT = '/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_all_data_2015_2025_tokenized.csv'
CHUNK_SIZE = 10000  # 可按内存情况调整，比如 5000/20000

start = time.time()

if not os.path.exists(INPUT):
    raise FileNotFoundError(f'找不到输入文件：{INPUT}')

# 可选的小清洗：去除多余空白/URL/HTML 标签（需要就留着，不需要可注释掉）
_url_pat  = re.compile(r'https?://\S+')
_html_pat = re.compile(r'<[^>]+>')

def clean_text(s: str) -> str:
    s = _url_pat.sub(' ', s)
    s = _html_pat.sub(' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def tokenize_vi(text):
    if pd.isna(text) or str(text).strip() == '':
        return ''
    # 如不想清洗，改为：txt = str(text)
    txt = clean_text(str(text))
    return ViTokenizer.tokenize(txt)

# 如果输出已存在，先删掉，避免重复追加
if os.path.exists(OUTPUT):
    os.remove(OUTPUT)

# 分块读取 + 处理 + 逐块写出（保留所有原列，并新增 content_tokenized）
reader = pd.read_csv(
    INPUT,
    chunksize=CHUNK_SIZE,
    dtype=str,                # 统一按字符串读入，避免类型冲突
    encoding_errors='ignore'  # 容忍少量坏字符
)

first_chunk = True
total_rows_in = 0
total_rows_out = 0

for chunk in tqdm(reader, desc='Tokenizing content', unit='块'):
    total_rows_in += len(chunk)

    # 确保有 content 列
    if 'content' not in chunk.columns:
        raise ValueError("没有找到 'content' 列，请检查输入 CSV。")

    # 分词
    chunk['content_tokenized'] = chunk['content'].apply(tokenize_vi)

    # 写出
    if first_chunk:
        chunk.to_csv(OUTPUT, index=False, mode='w')
        first_chunk = False
    else:
        chunk.to_csv(OUTPUT, index=False, mode='a', header=False)

    total_rows_out += len(chunk)

elapsed = time.time() - start
print(f'分词完成，保存到：{OUTPUT}')
print(f'输入行数：{total_rows_in:,}；输出行数：{total_rows_out:,}')
print(f'用时：{elapsed:.2f} 秒')


Tokenizing content: 1块 [00:10, 10.23s/块]

分词完成，保存到：/content/drive/MyDrive/Colab Notebooks/STAGE_CRLAO_CNRS/Corpus_data/Token/vi_all_data_2015_2025_tokenized.csv
输入行数：758；输出行数：758
用时：10.24 秒
